## 1. Import All Packages

In [4]:
%%time
import numpy as np
import math
import pandas as pd
import pickle, os, math
from keras.models import Sequential
from keras.callbacks import Callback
from keras.layers import Dense
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from tqdm import tqdm
from random import randint

Wall time: 0 ns


## 2. Read data

The data is combined from three different datasets.

The enermy frequency are respectively 2, 1, 8.

In [5]:
%%time

data_dir = os.path.join(os.curdir, 'Data', 'Score', '400000_68per', 'data_basic.pkl')
with open(data_dir, 'rb') as in_file:
    ot = pickle.load(in_file)
data_pics_0 = ot['data']
target_pics_0 = ot['target']
print(data_pics_0.shape)

data_dir = os.path.join(os.curdir, 'Data', 'Score', '400000_1freq_fires1', 'data_basic.pkl')
with open(data_dir, 'rb') as in_file:
    ot = pickle.load(in_file)
data_pics_3 = ot['data']
target_pics_3 = ot['target']
print(data_pics_3.shape)

data_dir = os.path.join(os.curdir, 'Data', 'Score', '800000_8freq', 'data_basic.pkl')
with open(data_dir, 'rb') as in_file:
    ot = pickle.load(in_file)
data_pics = ot['data']
target_pics = ot['target']
print(data_pics.shape)

data_pics = data_pics.append(data_pics_0).append(data_pics_3)
target_pics = target_pics.append(target_pics_0).append(target_pics_3)


data_pics.reset_index(inplace=True, drop=True)
target_pics.reset_index(inplace=True, drop=True)

print(data_pics.shape)

(90029, 221)
(41569, 221)
(599671, 221)
(731269, 221)
Wall time: 3.44 s


## 3. Preprocess Data

The same preprocessing is done to the scoring training data.

The imbalance between classes are more prominent, so a new group of data is generated using the same method. The new dataset has 82k data points, which should be enough for training purposes.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    data_pics, target_pics, test_size=0.1, random_state=randint(100, 10000))
print(X_train.shape)
X_train_train, X_vali, y_train_train, y_vali = train_test_split(
    X_train, y_train, test_size=0.3, random_state=randint(100, 10000))
print('class 0 has ' + str(len(y_train.index[y_train[0] == 0].tolist())) + ' points')
print('class 1 has ' + str(len(y_train.index[y_train[0] == 1].tolist())) + ' points')

index_0 = y_train.index[y_train[0] == 0].tolist()
index_1 = y_train.index[y_train[0] != 0].tolist()
index_0_comparable_to_1 = np.random.choice(index_0, math.floor(len(index_1) * 1))
samples = np.concatenate([index_1, index_0_comparable_to_1])
print(str(len(index_1)) + ' + ' + str(len(index_0_comparable_to_1)) + ' = ' + str(len(samples)))


small_data = data_pics.iloc[samples, :]
small_target = target_pics.iloc[samples, :]

X_train_small, X_test_small, y_train_small, y_test_small = train_test_split(
    small_data, small_target, test_size=0.3, random_state=randint(100, 10000))
X_train_train_small, X_vali_small, y_train_train_small, y_vali_small = train_test_split(
    X_train_small, y_train_small, test_size=0.3, random_state=randint(100, 10000))

y_train_small_m = np.ravel(y_train_small)
y_test_small_m = np.ravel(y_test_small)
y_train_train_small_m = np.ravel(y_train_train_small)
y_vali_small_m = np.ravel(y_vali_small)
y_train_m = np.ravel(y_train)
y_test_m = np.ravel(y_test)
y_train_train_m = np.ravel(y_train_train)
y_vali_m = np.ravel(y_vali)

(658142, 221)
class 0 has 616613 points
class 1 has 41529 points
41529 + 41529 = 83058


## 3. Training

### Multi Layer Perceptor Classifier

In [7]:
mlpc_st = MLPClassifier(hidden_layer_sizes=(256, 128, 64, 32, 16, 8),
                        alpha=0.15, max_iter=1000, batch_size=2000,
                        verbose=True, learning_rate_init=0.01, tol=1e-5,
                        learning_rate='adaptive')

mlpc_st.fit(X_train_small, y_train_small_m)
mlpc_st.score(X_test_small, y_test_small_m)

Iteration 1, loss = 0.71077937
Iteration 2, loss = 0.56123815
Iteration 3, loss = 0.54521919
Iteration 4, loss = 0.53765653
Iteration 5, loss = 0.52040314
Iteration 6, loss = 0.50798682
Iteration 7, loss = 0.49923229
Iteration 8, loss = 0.49477886
Iteration 9, loss = 0.48370096
Iteration 10, loss = 0.49336071
Iteration 11, loss = 0.47597587
Iteration 12, loss = 0.47064433
Iteration 13, loss = 0.46180174
Iteration 14, loss = 0.46382638
Iteration 15, loss = 0.46263809
Iteration 16, loss = 0.46192799
Training loss did not improve more than tol=0.000010 for two consecutive epochs. Stopping.


0.7807207641062686

In [8]:
ypred = mlpc_st.predict(X_test_small)
np.histogram(ypred - y_test_small_m + 1, bins=[0, 1, 2, 3])

(array([ 3076, 19454,  2388], dtype=int64), array([0, 1, 2, 3]))

In [9]:
cross_val_score(mlpc_st, X_train_small, y_train_small_m, cv=5)

Iteration 1, loss = 0.71261470
Iteration 2, loss = 0.55959339
Iteration 3, loss = 0.55142420
Iteration 4, loss = 0.54814739
Iteration 5, loss = 0.54696221
Iteration 6, loss = 0.54093755
Iteration 7, loss = 0.53758349
Iteration 8, loss = 0.53550233
Iteration 9, loss = 0.52066649
Iteration 10, loss = 0.51307636
Iteration 11, loss = 0.50353729
Iteration 12, loss = 0.49809278
Iteration 13, loss = 0.49306400
Iteration 14, loss = 0.48838948
Iteration 15, loss = 0.47812446
Iteration 16, loss = 0.47741668
Iteration 17, loss = 0.47061686
Iteration 18, loss = 0.47172241
Iteration 19, loss = 0.46214790
Iteration 20, loss = 0.46251245
Iteration 21, loss = 0.46349227
Iteration 22, loss = 0.45994913
Iteration 23, loss = 0.45249831
Iteration 24, loss = 0.44145668
Iteration 25, loss = 0.44242797
Iteration 26, loss = 0.43816604
Iteration 27, loss = 0.43772209
Iteration 28, loss = 0.43178104
Iteration 29, loss = 0.42987710
Iteration 30, loss = 0.42212287
Iteration 31, loss = 0.41446887
Iteration 32, los

Iteration 47, loss = 0.35438992
Iteration 48, loss = 0.34580684
Iteration 49, loss = 0.35849859
Iteration 50, loss = 0.35167317
Iteration 51, loss = 0.33723722
Iteration 52, loss = 0.33981456
Iteration 53, loss = 0.33964535
Iteration 54, loss = 0.34691387
Training loss did not improve more than tol=0.000010 for two consecutive epochs. Stopping.


array([0.75629891, 0.74191606, 0.75240798, 0.75068799, 0.756429  ])

In [10]:
import joblib
joblib.dump(mlpc_st, os.path.join(os.curdir,'Models', 'model_score_75_method_1.joblib'))
mlpc_st_loaded = joblib.load(os.path.join(os.curdir,'Models', 'model_score_75_method_1.joblib'))

In [11]:
mlpc_st_loaded.predict([[i for i in range(221)]])

array([1], dtype=int64)

### SVC with scikit-learn

To use SVC, we have to scale down the dataset.

In [12]:
index_0 = y_train.index[y_train[0] == 0].tolist()
index_1 = y_train.index[y_train[0] != 0].tolist()

index_1_svc = np.random.choice(index_1, math.floor(len(index_1) * 0.02))
index_0_svc = np.random.choice(index_0, math.floor(len(index_1_svc) * 1))
samples_svc = np.concatenate([index_1_svc, index_0_svc])
print(str(len(index_1_svc)) + ' + ' + str(len(index_0_svc)) + ' = ' + str(len(samples_svc)))


svc_data = data_pics.iloc[samples, :]
svc_target = target_pics.iloc[samples, :]

X_train_svc, X_test_svc, y_train_svc, y_test_svc = train_test_split(
    svc_data, svc_target, test_size=0.3, random_state=randint(100, 10000))
X_train_train_svc, X_vali_svc, y_train_train_svc, y_vali_svc = train_test_split(
    X_train_svc, y_train_svc, test_size=0.3, random_state=randint(100, 10000))

y_train_svc_m = np.ravel(y_train_svc)
y_test_svc_m = np.ravel(y_test_svc)
y_train_train_svc_m = np.ravel(y_train_train_svc)
y_vali_svc_m = np.ravel(y_vali_svc)

830 + 830 = 1660


In [13]:
clf_nf = SVC(C=1.0, gamma='auto', verbose=True)
clf_nf.fit(X_train_svc, y_train_svc_m)
clf_nf.score(X_test_svc, y_test_svc_m)

[LibSVM]

0.7592503411188699

## 4. Using predictors in game

### 4.1 without using survival training, 

<p style="text-align: center;">**enemy frequency: 1 / 1**</p>

Before Score training (6 test runs):

|                    |    1    |    2    |    3    |    4    |    5    |    6    |
|--------------------|--------:|--------:|--------:|--------:|--------:|--------:|
|Lives Consumed      | 2950.000| 2836.000| 2704.000| 2526.000| 2677.000| 2758.000|
|Score               | 1586.000| 1505.000| 1496.000| 1317.000| 1413.000| 1560.000|
|Fires               | 3652.000| 3743.000| 3851.000| 3709.000| 3653.000| 3762.000|
|Moves               |15001.000|15001.000|15001.000|15001.000|15001.000|15001.000|
|Ratio(Lives / Mov)  |    0.197|    0.189|    0.180|    0.168|    0.178|    0.184|
|Ratio(Score / Mov)  |    0.106|    0.100|    0.100|    0.088|    0.094|    0.104|
|Ratio(Score / Fires)|    0.434|    0.402|    0.388|    0.355|    0.387|    0.415|

After score training (6 test runs):

|                    |    1    |    2    |    3    |    4    |    5    |
|--------------------|--------:|--------:|--------:|--------:|--------:|
|Lives Consumed      | 1703.000| 1565.000| 1612.000| 1551.000| 1667.000|
|Score               | 1535.000| 1481.000| 1373.000| 1338.000| 1381.000|
|Fires               | 5413.000| 5291.000| 5252.000| 5048.000| 5122.000|
|Moves               |12001.000|12001.000|12001.000|12001.000|12001.000|
|Ratio(Lives / Mov)  |    0.142|    0.130|    0.134|    0.129|    0.139|
|Ratio(Score / Mov)  |    0.128|    0.123|    0.114|    0.111|    0.115|
|Ratio(Score / Fires)|    0.284|    0.280|    0.261|    0.265|    0.270|

The average score/fire ratio increased from 0.396 to 0.411, which is not very obvious.

<p style="text-align: center;">**enemy frequency: 1 / 4**</p>

Before Score training (5 test runs):

|                    |    1     |    2     |    3    |    4    |    5     |
|--------------------|---------:|---------:|--------:|--------:|---------:|
|Lives Consumed      |  492.0000|  444.0000|  544.000|  557.000|  555.0000|
|Score               |  291.0000|  289.0000|  319.000|  330.000|  294.0000|
|Fires               | 3056.0000| 2980.0000| 3044.000| 3044.000| 3054.0000|
|Moves               |12001.0000|12001.0000|12001.000|12001.000|12001.0000|
|Ratio(Lives / Mov)  |    0.0410|    0.0370|    0.045|    0.046|    0.0462|
|Ratio(Score / Mov)  |    0.0242|    0.0241|    0.027|    0.027|    0.0245|
|Ratio(Score / Fires)|    0.0952|    0.0970|    0.105|    0.108|    0.0963|

After score training (5 test runs):



There is no improvements at all.


<p style="text-align: center;">**enemy frequency: 1 / 8**</p>

Before Score training (5 test runs):

|                    |    1     |    2     |    3     |    4     |    5     |
|--------------------|---------:|---------:|---------:|---------:|---------:|
|Lives Consumed      |  186.0000|  272.0000|  293.0000|  294.0000|  282.0000|
|Score               |  124.0000|  188.0000|  146.0000|  172.0000|  167.0000|
|Fires               | 3071.0000| 2874.0000| 2966.0000| 2936.0000| 2988.0000|
|Moves               |12001.0000|12001.0000|12001.0000|12001.0000|12001.0000|
|Ratio(Lives / Mov)  |    0.0155|    0.0227|    0.0244|    0.0245|    0.0235|
|Ratio(Score / Mov)  |    0.0103|    0.0157|    0.0122|    0.0143|    0.0139|
|Ratio(Score / Fires)|    0.0404|    0.0654|    0.0492|    0.0586|    0.0559|

After score training (5 test runs):




### 4.1 Using survival training,

<p style="text-align: center;">**enemy frequency: 1 / 4**</p>

Before Score training:


After score training: